In [1]:
#import items

import json
import re

import requests
import scrapy

In [2]:
#goheels url
base_url = 'http://goheels.com'
#basketball roster
url = base_url + '/roster.aspx?path=baseball'
headers = {'User-Agent': 'UNC Journo Class'}
resp = requests.get(url, headers={'User-Agent': 'UNC Journo Class'},)

In [41]:
#getting body
body_bytes = resp.content
body_str = body_bytes.decode('utf-8')

In [42]:
#testing to see if it works
body_str

'\r\n\r\n<!doctype html>\r\n<html id="ctl00_html" lang="en" class=" wide baseball sport baseball"> \r\n    <head><script>window.client_hostname = "admin.goheels.com";window.server_name = "866411-LB16";window.client_division = "DI";window.targetting_criteria = {"page_template":"roster-list","sport_name":"baseball","sport_name_custom":"baseball","site":"unc","content_id":null,"sect":"mbasebl"};window.client_shortname = "unc";window.dfp_network_code = "29658103";window.img_dfp_unit_name = "";window.block_dfp = false;window.reactions = null;</script><link rel="stylesheet" href="/less/sidearm-responsive-grid.less?_=636553653061388688"><link rel="stylesheet" href="https://s3.amazonaws.com/assets.sidearmsports.com/adaptive/css/normalize.min.css"><link rel="apple-touch-icon" href="/images/logos/site/site.png"><link rel="canonical" href="http://goheels.com/roster.aspx?roster=498&amp;path=baseball"><meta charset="utf-8"><meta name="viewport" content="width=device-width"><link rel="apple-touch-ic

In [4]:
#we now want to select the info that we have collected in order to do stuff with it!
sel = scrapy.Selector(text=body_str)

In [5]:
#getting the table
table = sel.css('table')[0]

In [7]:
table.css('tr') # Get all the tr elements within the table
table.css('tr')[0] # Get the first tr element within the table using the pythonic index
table.css('tr')[0].css('th') # Get all the th elements from within the first tr
table.css('tr:nth-child(1) th')

[<Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' 

In [8]:
#column names
columns = table.css('th').xpath('string()').extract()

In [43]:
columns

['#',
 'Full Name',
 'Pos.',
 'Ht.',
 'Wt.',
 'Academic Year',
 'Hometown / High School']

In [10]:
#rest of rows
rows = table.css('tr')[1:]

In [11]:
#creating new list for each player
by_player_number = []
for r in rows:
    #dictionary called data
    data = {}
    for i, d in enumerate(r.css('td')): #enumerate goes on to next item
        a = d.css('a')
        if a:
            t = a.xpath('text()').extract()[0]
            data['href'] = a.xpath('@href').extract()[0]
        else:
            t = d.xpath('text()').extract()[0]
        data[columns[i]] = t
    by_player_number.append(data)

In [12]:
by_player_number 

[{'#': '1',
  'Academic Year': 'Jr.',
  'Full Name': 'Brandon Riley',
  'Hometown / High School': 'Burlington, N.C. / Williams',
  'Ht.': '6-0',
  'Pos.': 'OF',
  'Wt.': '175',
  'href': '/roster.aspx?rp_id=14221'},
 {'#': '2',
  'Academic Year': 'Fr.',
  'Full Name': 'Satchel Jerzembeck',
  'Hometown / High School': 'Charlotte, N.C. / Providence',
  'Ht.': '5-10',
  'Pos.': 'IF',
  'Wt.': '150',
  'href': '/roster.aspx?rp_id=14226'},
 {'#': '3',
  'Academic Year': 'Jr.',
  'Full Name': 'Kyle Datres',
  'Hometown / High School': 'Williamsport, Pa. / Loyalsock Township',
  'Ht.': '6-0',
  'Pos.': '3B',
  'Wt.': '198',
  'href': '/roster.aspx?rp_id=14211'},
 {'#': '4',
  'Academic Year': 'So.',
  'Full Name': 'Brandon Martorano',
  'Hometown / High School': 'Marlboro, N.J. / Christian Brothers Academy',
  'Ht.': '6-2',
  'Pos.': 'C/OF',
  'Wt.': '187',
  'href': '/roster.aspx?rp_id=14219'},
 {'#': '5',
  'Academic Year': 'So.',
  'Full Name': 'Ashton McGee',
  'Hometown / High School': '

In [13]:
#getting each player bio
def fetch_bio(player):
    #goheels.com + player url
    player_url = base_url + player['href']
    print('Fetch bio', player_url)
    resp = requests.get(player_url, headers=headers)
    player_txt = resp.content.decode('utf-8')
    sel = scrapy.Selector(text=player_txt)
    player['sel'] = sel
    player['bio'] = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    player['img'] = sel.css('.sidearm-roster-player-image img').xpath('@src').extract()[0]

In [14]:
#regex
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?(?P<obj>{.*?})')

In [15]:
#loop to fetch stats
def fetch_stats(player):
    text = player['sel'].xpath('string()').extract()[0]
    parts = text.split('$.getJSON("/services/')[1:]
    captured = js_obj_rx.findall(''.join(parts))
    clean_objs = []
    for obj_str in captured:
        if 'stats' not in obj_str:
            continue

        obj_str = obj_str.replace('{', '').replace('}', '')
        obj_str = obj_str.replace("'", '').replace('"', '')
        obj_pairs = obj_str.split(',')
        obj_pairs = [x.split(":") for x in obj_pairs]
        clean_pairs = []
        for pair in obj_pairs:
            clean_pairs.append(['"{}"'.format(p.strip()) for p in pair])
        colonized = [":".join(p) for p in clean_pairs]
        commas = ','.join(colonized)
        json_str = "{" + commas + "}"
        clean_objs.append(json.loads(json_str))
    
    player['stats_url'] = stats_url = (
        "http://goheels.com/services/responsive-roster-bio.ashx?"
        "type={type}&rp_id={rp_id}&path={path}&year={year}"
        "&player_id={player_id}"
    ).format(**clean_objs[0])
    
    print('Fetch stats', stats_url)
    resp = requests.get(stats_url, headers=headers)
    json_stats = json.loads(resp.content.decode("utf-8"))
    player['raw_stats'] = json_stats

In [16]:
#getting bio and stats
for p in by_player_number:
    fetch_bio(p)
    fetch_stats(p)

Fetch bio http://goheels.com/roster.aspx?rp_id=14221
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14221&path=baseball&year=2018&player_id=3746
Fetch bio http://goheels.com/roster.aspx?rp_id=14226
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14226&path=baseball&year=2018&player_id=0
Fetch bio http://goheels.com/roster.aspx?rp_id=14211
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14211&path=baseball&year=2018&player_id=3736
Fetch bio http://goheels.com/roster.aspx?rp_id=14219
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14219&path=baseball&year=2018&player_id=3760
Fetch bio http://goheels.com/roster.aspx?rp_id=14220
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14220&path=baseball&year=2018&player_id=3761
Fetch bio http://goheels.com/roster.aspx?rp_id=14228
Fetch stats http://goheels.com/services/

In [17]:
#getting first player in roster 
by_player_number[0]

{'#': '1',
 'Academic Year': 'Jr.',
 'Full Name': 'Brandon Riley',
 'Hometown / High School': 'Burlington, N.C. / Williams',
 'Ht.': '6-0',
 'Pos.': 'OF',
 'Wt.': '175',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with a homer 

In [44]:
p = [p for p in by_player_number if p['Full Name'] == 'Brandon Riley'][0]

In [45]:
#raw and career stats
txt = p['raw_stats']['career_stats']
#print it
print(txt)


            <section>
                <h5>Hitting Statistics</h5>

                <div class="sidearm-table-overflow-on-x-large">
                    <table class="sidearm-table highlight-column-hover">
                        <caption>Career Hitting Statistics</caption>
                        <thead>
                            <tr>
                                <th scope="col" class="text-hide">Season</th>
                                <th scope="col">AVG</th>
                                <th scope="col">GP</th>
                                <th scope="col">GS</th>
                                <th scope="col">AB</th>
                                <th scope="col">R</th>
                                <th scope="col">H</th>
                                <th scope="col">2B</th>
                                <th scope="col">3B</th>
                                <th scope="col">HR</th>
                                <th scope="col">RBI</th>
                       

In [46]:
sel = scrapy.Selector(text=txt)

In [47]:
#getting stats section
sel.css('section')

[<Selector xpath='descendant-or-self::section' data='<section>\r\n                <h5>Hitting S'>]

In [48]:
#cleaning stats to be readable 
def parse_stats(player):
    stats = {}
    for raw_key, raw_val in player['raw_stats'].items():
        txt = player['raw_stats'][raw_key]
        if not txt:
            print('Skipping {} for {}'.format(raw_key, player['Full Name']))
            continue
        sel = scrapy.Selector(text=txt)
        # Get all the tables
        for section in sel.css('section'):
            title = section.css('h5').xpath('string()').extract()[0]
            cols = section.css('tr')[0].css('th').xpath('string()').extract()
            print('TITLE OF STATS TABLE', title)
            print('COLUMNS', cols)
            these_stats = []
            print('ROWS', section.css('tr'))
            for r in section.css('tr')[1:]:
                print('row', r.xpath('string()').extract()[0].replace('\r', '').replace('\n', '').strip())
                s = {}
                for i, d in enumerate(r.css('td'), 1):
                    s[cols[i].lower()] = d.xpath('string()').extract()[0]
                yr = r.css('th').xpath('string()')
                if yr:
                    yr = yr.extract()[0]
                    if yr.lower() in ('total', 'season'):
                        print('SKIPPING...')
                        continue
                    print('THE YEAR IS', yr)
                    s['yr'] = yr
                these_stats.append(s)
                print('THE STATS ARE', these_stats)
            existing = stats.get(raw_key, {})
            existing[title] = these_stats
            stats[raw_key] = existing
    player['stats'] = stats

In [49]:
#parsing stats for the first player to see if it works
p = [p for p in by_player_number if p['Full Name'] == 'Brandon Riley'][0]
parse_stats(p)

TITLE OF STATS TABLE Hitting Statistics
COLUMNS ['Date', 'Opponent', 'W/L', 'GS', 'AB', 'R', 'H', 'RBI', '2B', '3B', 'HR', 'BB', 'IBB', 'SB', 'SBA', 'CS', 'HBP', 'SH', 'SF', 'GDP', 'K', 'AVG']
ROWS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t<th scope="col" class="tex'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/16/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/17/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/18/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/20/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/21/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/23/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data=

IndexError: list index out of range

In [50]:
#everyone else's stats
for p in by_player_number:
    parse_stats(p)

TITLE OF STATS TABLE Hitting Statistics
COLUMNS ['Date', 'Opponent', 'W/L', 'GS', 'AB', 'R', 'H', 'RBI', '2B', '3B', 'HR', 'BB', 'IBB', 'SB', 'SBA', 'CS', 'HBP', 'SH', 'SF', 'GDP', 'K', 'AVG']
ROWS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t<th scope="col" class="tex'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/16/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/17/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/18/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/20/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/21/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n\t\t\t\t\t\t\t\t\t<td>2/23/2018</td>\r\n\t\t\t\t\t'>, <Selector xpath='descendant-or-self::tr' data=

IndexError: list index out of range

In [51]:
print(p)
temp_player = [p for p in by_player_number if p['Full Name'] == 'Brandon Riley'][0]

print(temp_player)
#AND WHY IS 'STATS' A KEY ERROR - ITS DEFINED ABOVE

{'#': '1', 'href': '/roster.aspx?rp_id=14221', 'Full Name': 'Brandon Riley', 'Pos.': 'OF', 'Ht.': '6-0', 'Wt.': '175', 'Academic Year': 'Jr.', 'Hometown / High School': 'Burlington, N.C. / Williams', 'sel': <Selector xpath=None data='<html id="ctl00_html" lang="en" class=" '>, 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed

In [52]:
by_player_number

[{'#': '1',
  'Academic Year': 'Jr.',
  'Full Name': 'Brandon Riley',
  'Hometown / High School': 'Burlington, N.C. / Williams',
  'Ht.': '6-0',
  'Pos.': 'OF',
  'Wt.': '175',
  'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with 

In [53]:
[p for p in by_player_number if p['Full Name'] == 'Brandon Riley'][0]['stats']

KeyError: 'stats'

In [54]:
#creating a new json file and dumping data into it
to_dump = [p.copy() for p in by_player_number]
for p in to_dump:
    p.pop('sel')
    for k in list(p.keys()):
        if 'raw' in k:
            p.pop(k)
with open('bball_scraped.json', 'w') as f:
    json.dump(to_dump, f)

In [55]:
cat bball_scraped.json | cut -c 1-100

[{"#": "1", "href": "/roster.aspx?rp_id=14221", "Full Name": "Brandon Riley", "Pos.": "OF", "Ht.": "


In [56]:
#what was dumped for the first player
to_dump[0]

{'#': '1',
 'Academic Year': 'Jr.',
 'Full Name': 'Brandon Riley',
 'Hometown / High School': 'Burlington, N.C. / Williams',
 'Ht.': '6-0',
 'Pos.': 'OF',
 'Wt.': '175',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with a homer 

In [57]:
import numpy as np
import pandas as pd

In [58]:
data = json.load(open('bball_scraped.json'))

In [59]:
#data for first player
data[0]

{'#': '1',
 'Academic Year': 'Jr.',
 'Full Name': 'Brandon Riley',
 'Hometown / High School': 'Burlington, N.C. / Williams',
 'Ht.': '6-0',
 'Pos.': 'OF',
 'Wt.': '175',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with a homer 

In [60]:
#data frame
data_for_future_df = []

In [61]:
for player in data:
    for key, val in player.get('stats', {}).get('career_stats', {}).items():
        stat_type = key.lower().split()[0]
        name = player.get('Full Name')
        if not name:
            continue
        num = player.get('#')
        if not num:
            continue
            
        for stat in val:
            stat['name'] = name
            stat['num'] = num
            stat['stat_type'] = stat_type
            data_for_future_df.append(stat)

In [62]:
#data frame variable
df = pd.DataFrame(data_for_future_df)

In [63]:
#df header 
df.head()

""


In [64]:
df = df.apply(pd.to_numeric, errors='ignore')

In [65]:
df.drop('avg', axis=1, inplace=True)

ValueError: labels ['avg'] not contained in axis

In [66]:
df.head()

""


In [67]:
#Player batting average 
df['ba'] = df.h / df.ab

AttributeError: 'DataFrame' object has no attribute 'h'

In [68]:
# grouping 
by_player = df.groupby(('name', 'stat_type'))
by_year = df.groupby('yr')
by_type = df.groupby('stat_type')

KeyError: 'name'

In [69]:
#sorting
by_type.get_group('hitting').groupby('yr').mean().ba

NameError: name 'by_type' is not defined

In [70]:
by_player.mean().h / by_player.mean().ab

NameError: name 'by_player' is not defined

In [71]:
df.loc[:, ('name', 'yr', 'ba')]

KeyError: "None of [('name', 'yr', 'ba')] are in the [columns]"